In [7]:
import pandas as pd
import os
import glob

path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "data/*.csv"))

In [17]:
dataframes = []

for f in csv_files:

    # read the csv file
    df = pd.read_csv(f)
    
    # skip setup files
    if len(df) < 2:
        continue
    
    # add UUID as column
    df["participantUuid"] = os.path.splitext(f)[0][-36:]
    
    # reformat timestamp for Excel
    df['timestamp'] = df['timestamp'].astype('datetime64[s]')
    
    # append to array
    dataframes.append(df)

In [18]:
# concatenate data frames
merged = pd.concat(dataframes)

# save to svg
merged.to_csv("merged.csv")